# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
import requests
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display


In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')


In [5]:
system_prompt = """
    You are a customer service representative for a company that
    answers technical question. A customer has contacted you with a question
    about the product."
"""

In [6]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [13]:
def stream_response(openai, model: str, system_prompt: str, question: str):
    stream = openai.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
        ],
    stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [14]:
# Get gpt-4o-mini to answer, with streaming
openai = OpenAI()
stream_response(openai, MODEL_GPT, system_prompt, question)

The code you've provided is a generator expression that produces a sequence of authors from a collection of books, specifically from a list or iterable named `books`. Let's break it down:

1. **Set Comprehension**: The code `{book.get("author") for book in books if book.get("author")}` is a set comprehension. This creates a set of authors:
   - `book.get("author")` retrieves the value associated with the key `"author"` for each `book` in the `books` collection.
   - The condition `if book.get("author")` ensures that only books with an author value (i.e., the author's name is not `None`, `False`, or an empty string) are included in the set.

2. **Yield from**: The `yield from` keyword is used in a generator function. It allows the generator to yield all values from the iterable (in this case, the set created by the set comprehension) as if they were being produced one by one.
   - Because this is a generator, the values can be consumed lazily, meaning that they are computed one at a time and only as needed.

### Summary:
- **Function**: The entire expression extracts unique authors from a list of books while filtering out any entries without an author, and then yields these authors one at a time.
- **Why**: Using a generator here with `yield from` is efficient for handling potentially large datasets, as it allows for processing elements sequentially without loading them all into memory at once. Additionally, using a set ensures that each author is only returned once, eliminating duplicates.

This can be particularly useful in scenarios where you want to enumerate or process authors without accumulating them all at once, thus keeping memory usage lower.

In [15]:
# Get Llama 3.2 to answer
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
stream_response(ollama_via_openai, MODEL_LLAMA, system_prompt, question)

This line of code uses a combination of generator expression, dictionary lookup, and conditional filtering to extract author names from a list of dictionaries (`books`).

Here's a breakdown:

- `for book in books`: This part iterates over each dictionary in the `books` list.
- `{book.get("author") for book in books if book.get("author")}`: This is a generator expression that filters out dictionaries without an "author" key and yields only the author name from dictionaries with an "author" key.

  - `for book in books` iterates over each dictionary (`book`) in `books`.
  - `if book.get("author")`: This condition checks if the current dictionary has a key named "author". If it does, it proceeds to the next step.
  
- `yield from {...}`: The `yield from` clause is used to yield results from another iterable (in this case, the generator expression). When `yield from` is encountered in a function definition or class definition, it causes a new execution context for that part of code. The returned values are yielded by the generator expression defined on the right side (`{...}`).

In summary, this line yields the author name of each book that has an "author" key without actually loading all books into memory at once.

Here's how you might use this in a loop to yield authors one at a time:

python
for book in books:
    if book.get("author"):
        for author in yield from {book.get("author") for book in books if book.get("author")}:
            print(author)


This code will keep printing the authors of all books until the loop is stopped, instead of printing them all at once which would require more memory than available.

However, this approach still has a time complexity of O(n^2) because it uses dictionary lookup within its own generator. If you're dealing with large lists and/or dictionaries, there may be performance improvements if you can use other approaches like list comprehension with `if-else` or dictionary comprehension that handles nested dictionaries directly.

The following code snippet will have much better time complexity:

python
authors = {}
for book in books:
    author = book.get("author")
    if author is not None and author not in authors:
        yield author, {book["title"] for book in [b for b in books if (b["title"] == book["title" and "author"] == author]]}

authors_dict = dict(authors)
# use authors_dict here
